In [2]:
import os
from dotenv import load_dotenv,find_dotenv
from InstructorEmbedding import INSTRUCTOR
from langchain_groq import ChatGroq

from langchain.document_loaders import UnstructuredURLLoader
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA,  ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory


_ = load_dotenv(find_dotenv())
llm = ChatGroq(
        groq_api_key=os.environ.get("GROQ_API_KEY"),
        model_name="llama3-8b-8192"
)

def load_db():
    """
    loading data from source

    return a conversational retrieval Chain
    """
    
    # load data
    urls = ["https://gradspace.org/","https://gradspace.org/courses"]
    loaders = UnstructuredURLLoader(urls = urls)
    documents = loaders.load()
    
    # splitting data
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
    docs = text_splitter.split_documents(documents)
    
    # load Instructor Embeddings
    embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-base")
    
    # create vector database
    db = DocArrayInMemorySearch.from_documents(docs, embeddings)

    
    # define the retriever
    retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 3})

    
    # create chatbot
    qa = ConversationalRetrievalChain.from_llm(
        llm=llm, 
        chain_type="stuff", 
        retriever=retriever, 
        return_source_documents=True,
        return_generated_question=True,
    )
    return qa 




C:\Users\Stephen\AppData\Roaming\Python\Python312\site-packages\InstructorEmbedding\instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


In [3]:
# import os
# from dotenv import load_dotenv,find_dotenv
# from langchain_groq import ChatGroq
# from langchain.chains import LLMChain   
# from langchain.prompts import ChatPromptTemplate 
# from langchain.chains.router import MultiPromptChain 
# from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
# from langchain.prompts import PromptTemplate

# # load the llm
# _ = load_dotenv(find_dotenv())
# llm = ChatGroq(
#         groq_api_key=os.environ.get("GROQ_API_KEY"),
#         model_name="llama3-8b-8192"
# )

# destination_chains = {}
# for p_info in prompt_infos:
#     if name=="receptionist":
#         chain = load_db()
#     else:
#         name = p_info["name"]
#         prompt_template = p_info["prompt_template"]
#         prompt = ChatPromptTemplate.from_template(template=prompt_template)
#         chain = LLMChain(llm=llm, prompt=prompt)
#     destination_chains[name] = chain  
    
# destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
# destinations_str = "\n".join(destinations)

# default_prompt = ChatPromptTemplate.from_template("{input}")
# default_chain = LLMChain(llm=llm, prompt=default_prompt)


# MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
# language model select the model prompt best suited for the input. \
# You will be given the names of the available prompts and a \
# description of what the prompt is best suited for. \
# You may also revise the original input if you think that revising\
# it will ultimately lead to a better response from the language model.

# << FORMATTING >>
# Return a markdown code snippet with a JSON object formatted to look like:
# ```json
# {{{{
#     "destination": string \ name of the prompt to use or "DEFAULT"
#     "next_inputs": string \ a potentially modified version of the original input
# }}}}
# REMEMBER: "destination" MUST be one of the candidate prompt
# names specified below OR it can be "DEFAULT" if the input is not
# well suited for any of the candidate prompts. REMEMBER: "next_inputs" can just be the original input
# if you don't think any modifications are needed.

# << CANDIDATE PROMPTS >> {destinations}

# << INPUT >> {{input}}

# << OUTPUT (remember to include the ```json)>>"""



# router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format( destinations=destinations_str )

# router_prompt = PromptTemplate( template=router_template, input_variables=["input"], output_parser=RouterOutputParser(), )

# router_chain = LLMRouterChain.from_llm(llm, router_prompt)

# chain = MultiPromptChain(router_chain=router_chain, destination_chains=destination_chains, default_chain=default_chain, verbose=True )


In [4]:
# chain.run({"question": "what are the courses about backend development?","chat_history":"" , "input":"what are the courses about backend development?"})

In [5]:
import panel as pn
import param

# create a chatbot class
class cbfs(param.Parameterized):
    pn.extension()
    chat_history = param.List([])
    answer = param.String("")
    db_query  = param.String("")
    db_response = param.List([])
    
    def __init__(self,  **params):
        super(cbfs, self).__init__( **params)
        self.panels = []
        self.qa = load_db()
    

    #  load the conversational retrieval Chain, and store the results
    def convchain(self, query):
        """
        query
        """
        if not query:
            return pn.WidgetBox(pn.Row('User:', pn.pane.Markdown("", width=600)), scroll=True)
        result = self.qa({"question": query, "chat_history": self.chat_history})
        self.chat_history.extend([(query, result["answer"])])
        self.db_query = result["generated_question"]
        self.db_response = result["source_documents"]
        self.answer = result['answer'] 
        self.panels.extend([
            pn.Row('User:', pn.pane.Markdown(query, width=600)),
            pn.Row('ChatBot:', pn.pane.Markdown(self.answer, width=600))
        ])
        inp.value = ''  
        return pn.WidgetBox(*self.panels,scroll=True)

    
    # get the lastest query
    @param.depends('db_query ', )
    def get_lquest(self):
        if not self.db_query :
            return pn.Column(
                pn.Row(pn.pane.Markdown(f"Last question to DB:", styles={'background-color': '#F6F6F6'})),
                pn.Row(pn.pane.Str("no DB accesses so far"))
            )
        return pn.Column(
            pn.Row(pn.pane.Markdown(f"DB query:", styles={'background-color': '#F6F6F6'})),
            pn.pane.Str(self.db_query )
        )
    
    # get the lastest meta from database
    @param.depends('db_response', )
    def get_sources(self):
        if not self.db_response:
            return 
        rlist=[pn.Row(pn.pane.Markdown(f"Result of DB lookup:", styles={'background-color': '#F6F6F6'}))]
        for doc in self.db_response:
            rlist.append(pn.Row(pn.pane.Str(doc)))
        return pn.WidgetBox(*rlist, width=600, scroll=True)

    # get the chat history
    @param.depends('convchain', 'clr_history') 
    def get_chats(self):
        if not self.chat_history:
            return pn.WidgetBox(pn.Row(pn.pane.Str("No History Yet")), width=600, scroll=True)
        rlist=[pn.Row(pn.pane.Markdown(f"Current Chat History variable", styles={'background-color': '#F6F6F6'}))]
        for exchange in self.chat_history:
            rlist.append(pn.Row(pn.pane.Str(exchange)))
        return pn.WidgetBox(*rlist, width=600, scroll=True)
    
    # clear the chat history
    def clr_history(self,count=0):
        self.chat_history = []
        return 


In [6]:
# initialise the chatbot
cb = cbfs() 

# create the panel

button_load = pn.widgets.Button(name="Load DB", button_type='primary') 
button_clearhistory = pn.widgets.Button(name="Clear History", button_type='warning') 
button_clearhistory.on_click(cb.clr_history) 
inp = pn.widgets.TextInput( placeholder='Enter text here…') 


# bind the panel button with conversational retrieval Chain 
conversation = pn.bind(cb.convchain, inp) 

# jpg_pane = pn.pane.Image( './img/convchain.jpg')

# define the interface panel
tab1 = pn.Column(
    pn.Row(inp),
    pn.layout.Divider(),
    pn.panel(conversation,  loading_indicator=True, height=300),
    pn.layout.Divider(),
)
tab2= pn.Column(
    pn.panel(cb.get_lquest),
    pn.layout.Divider(),
    pn.panel(cb.get_sources ),
)
tab3= pn.Column(
    pn.panel(cb.get_chats),
    pn.layout.Divider(),
)


dashboard = pn.Column(
    pn.Row(pn.pane.Markdown('# ChatWithYourData_Bot')),
    pn.Tabs(('Conversation', tab1), ('Database', tab2), ('Chat History', tab3))
)
dashboard.servable()


load INSTRUCTOR_Transformer


C:\Users\Stephen\AppData\Roaming\Python\Python312\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
C:\Users\Stephen\AppData\Roaming\Python\Python312\site-packages\sentence_transformers\models\Dense.py:63: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed

max_seq_length  512


C:\Users\Stephen\AppData\Roaming\Python\Python312\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


Column
    [0] Row
        [0] Markdown(str)
    [1] Tabs
        [0] Column
            [0] Row
                [0] TextInput(placeholder='Enter text here…')
            [1] Divider()
            [2] ParamFunction(function, _pane=WidgetBox, defer_load=False, height=300, loading_indicator=True)
            [3] Divider()
        [1] Column
            [0] ParamMethod(method, _pane=Column, defer_load=False)
            [1] Divider()
            [2] ParamMethod(method, _pane=Str, defer_load=False)
        [2] Column
            [0] ParamMethod(method, _pane=WidgetBox, defer_load=False)
            [1] Divider()